# Processing the US EPA's 2019 National Emissions Inventory

This notebook contains a work-in-progress effort for processing the 2019 NEI using the [Julia language](https://julialang.org/).

First, we need to load the packages that we will use:

In [1]:
using Revise
using CSV
using ZipFile
using DataFrames
using DBFTables
using HTTP
using Unitful
using FilePathsBase
using Proj4
using Shapefile
using JSON
using GeometryBasics
using LibGEOS
using GeoInterface
using GeoFormatTypes
using SparseArrays
using Printf
using Plots

### Set download directory

Next, we need to choose a directory on our computer to download the emissions data to. You will probably need to change the directory below to a location that exists on your computer

In [3]:
dir = "/Users/$(ENV["USER"])/data/2019nei"

"C:/Users/hemamipo/OneDrive - University of Illinois - Urbana/Research-UIUC/Tessum Group/Tasks/NEI_Processor/2019nei"

## Download emissions files from the EPA web server

In this step, we download and unzip all of the emissions files.

You only need to do this once.

In [ ]:
urls = [
    "https://gaftp.epa.gov/Air/emismod/2019/2019emissions/2019ge_CMV_c1c2_inventory_16mar2023.zip",
    "https://gaftp.epa.gov/Air/emismod/2019/2019emissions/2019ge_CMV_c3_inventory_16mar2023.zip",
    "https://gaftp.epa.gov/Air/emismod/2019/2019emissions/2019ge_cem_inventory_10mar2023.zip",
    "https://gaftp.epa.gov/Air/emismod/2019/2019emissions/2019ge_fertilizer_inventory_04apr2023.zip",
    "https://gaftp.epa.gov/Air/emismod/2019/2019emissions/2019ge_fires_inventory_10mar2023.zip",
    "https://gaftp.epa.gov/Air/emismod/2019/2019emissions/2019ge_nonpoint_inventory_10mar2023.zip",
    "https://gaftp.epa.gov/Air/emismod/2019/2019emissions/2019ge_nonroad_inventory_10mar2023.zip",
    "https://gaftp.epa.gov/Air/emismod/2019/2019emissions/2019ge_onroad_SMOKE-MOVES_emissions_FF10_allHAPs_30nov2021_v0.zip",
    "https://gaftp.epa.gov/Air/emismod/2019/2019emissions/2019ge_onroad_activity_10mar2023.zip",
    "https://gaftp.epa.gov/Air/emismod/2019/2019emissions/2019ge_oth_inventory_17mar2023.zip",
    "https://gaftp.epa.gov/Air/emismod/2019/2019emissions/2019ge_point_inventory_10mar2023.zip",
    "https://gaftp.epa.gov/Air/emismod/2019/ancillary_data/2019ge_gridding_ge_dat_10mar2023.zip"
]

for url in urls
    println("Downloading $(url)")
    http_response = HTTP.get(url, require_ssl_verification = false)
    r = ZipFile.Reader(IOBuffer(http_response.body));
    for f in r.files
        println("Filename: $(f.name)")
        file_path = joinpath(dir, f.name)
        if f.name[end] == '/'
            mkpath(file_path)
            continue
        end
        mkpath(dirname(file_path))
        open(file_path, "w") do o
            write(o, read(f))
        end
    end
end

### Utility functions

This next cell contains some code that we will use to load and process the emissions. It will eventually be moved into a separate software library.

In [4]:
const tonperyear = 907.185u"kg" / 31_536_000u"s"
const tonpermonth = 907.185u"kg" / 2_628_288u"s"
const foot = (1/3.28084)u"m"
kelvin(F) = ((F − 32.0) * 5.0/9.0 + 273.15) * u"K"

abstract type EmissionsDataFrame end

# SurrogateSpec holds surrogate specification data
struct SurrogateSpec
    Region::String
    Name::String
    Code::Int
    DataShapefile::String
    DataAttribute::String
    WeightShapefile::String
    Details::String
    BackupSurrogateNames::Vector{String}
    WeightColumns::Vector{String}
    WeightFactors::Vector{Float64}
    FilterFunction::String
    MergeNames::Vector{String}
    MergeMultipliers::Vector{Float64}
end

# GridDef specifies the grid that we are allocating the emissions to.
struct GridDef
    Name::String
    Nx::Int
    Ny::Int
    SR::String
    Cells::Vector{LibGEOS.Polygon}
    Extent::Vector{Vector{Tuple{Float64, Float64}}}
end

# SpatialProcessor spatializes emissions records.
struct SpatialProcessor
    SrgSpecs::Vector{SurrogateSpec}
    Grids::GridDef
    GridRef::DataFrame
    InputSR::String
    MatchFullSCC::Bool
    MemCacheSize::Int
    MaxMergeDepth::Int
end

# Config holds configuration data.
struct Config
    f_gridRef::Vector{String}
    SrgSpec::String
    SrgShapefileDirectory::String
    InputSR::String
    OutputSR::String
    GridFile::String
    GridName::String
    Counties::String
    EmisShp::String
end

# IndexInfo holds grid index information for gridded emissions.
struct IndexInfo
    rows::Vector{Int}
    cols::Vector{Int}
    fracs::Vector{Float64}
    inGrid::Bool
    coveredByGrid::Bool
end

# Pollutants holds pollutant groups.
Pollutants = Dict(
    "EVP__VOC" => "VOC", "EXH__VOC" => "VOC", "VOC" => "VOC", "VOC_INV" => "VOC",
    "NOX" => "NOX",
    "NH3" => "NH3",
    "SO2" => "SO2",
    "BRK__PM25-PRI" => "PM25", "EXH__PM25-PRI" => "PM25", "EXH__PM2_5" => "PM25", 
    "PM25-PRI" => "PM25", "PM25TOTAL" => "PM25", "PM2_5" => "PM25", "TIR__PM25-PRI" => "PM25",
    "BRK__PM2_5" => "PM25", "TIR__PM2_5" => "PM25",
)

# https://www.cmascenter.org/smoke/documentation/4.8.1/html/ch08s02s04.html#d0e38214
struct FF10NonPointDataFrame <: EmissionsDataFrame
    df::DataFrame
    
    FF10NonPointDataFrame(df::DataFrame) = begin
        if size(df, 2) != 45
            throw(DimensionMismatch("FF10 nonpoint file should have 45 fields but instead has $(size(df,2))"))
        end
        
        rename!(df, ["COUNTRY", "FIPS", "TRIBAL_CODE", "CENSUS_TRACT", "SHAPE_ID", "SCC",
            "EMIS_TYPE", "POLID", "ANN_VALUE",
            "ANN_PCT_RED", "CONTROL_IDS", "CONTROL_MEASURES", "CURRENT_COST", "CUMULATIVE_COST", "PROJECTION_FACTOR",
            "REG_CODES", "CALC_METHOD", "CALC_YEAR", "DATE_UPDATED", "DATA_SET_ID", "JAN_VALUE", "FEB_VALUE", "MAR_VALUE",
            "APR_VALUE", "MAY_VALUE", "JUN_VALUE", "JUL_VALUE", "AUG_VALUE", "SEP_VALUE", "OCT_VALUE", "NOV_VALUE", "DEC_VALUE",
            "JAN_PCTRED", "FEB_PCTRED", "MAR_PCTRED", "APR_PCTRED", "MAY_PCTRED", "JUN_PCTRED", "JUL_PCTRED", "AUG_PCTRED",
            "SEP_PCTRED", "OCT_PCTRED", "NOV_PCTRED", "DEC_PCTRED", "COMMENT"])
        
        df[!, :FIPS] = begin
            fips_transformed = String[]
            for fips in df[!, :FIPS]
                fips_str = string(fips)
                if length(fips_str) == 6
                    fips_str = fips_str[2:end]
                end
                push!(fips_transformed, lpad(fips_str, 5, '0'))
            end
            fips_transformed
        end

        df[!, :SCC] = [lpad(scc, 10, '0') for scc in string.(df[!, :SCC])]
        
        df.ANN_VALUE = df.ANN_VALUE * tonperyear
        
        df.JAN_VALUE = df.JAN_VALUE * tonpermonth
        df.FEB_VALUE = df.FEB_VALUE * tonpermonth
        df.MAR_VALUE = df.MAR_VALUE * tonpermonth
        df.APR_VALUE = df.APR_VALUE * tonpermonth
        df.MAY_VALUE = df.MAY_VALUE * tonpermonth
        df.JUN_VALUE = df.JUN_VALUE * tonpermonth
        df.JUL_VALUE = df.JUL_VALUE * tonpermonth
        df.AUG_VALUE = df.AUG_VALUE * tonpermonth
        df.SEP_VALUE = df.SEP_VALUE * tonpermonth
        df.OCT_VALUE = df.OCT_VALUE * tonpermonth
        df.NOV_VALUE = df.NOV_VALUE * tonpermonth
        df.DEC_VALUE = df.DEC_VALUE * tonpermonth
        
        return new(df)
    end
end

struct FF10NonRoadDataFrame <: EmissionsDataFrame
    df::DataFrame
    
    FF10NonRoadDataFrame(df::DataFrame) = new(FF10NonPointDataFrame(df).df)
end

struct FF10OnRoadDataFrame <: EmissionsDataFrame
    df::DataFrame
    
    FF10OnRoadDataFrame(df::DataFrame) = new(FF10NonPointDataFrame(df).df)
end

# https://www.cmascenter.org/smoke/documentation/4.8.1/html/ch08s02s08.html#sect_input_ptinv_ff10
struct FF10PointDataFrame <: EmissionsDataFrame
    df::DataFrame
    
    FF10PointDataFrame(df::DataFrame) = begin
        if size(df, 2) != 77
            throw(DimensionMismatch("FF10 point file should have 77 fields but instead has $(size(df,2))"))
        end
        
        rename!(df, [
            "COUNTRY","FIPS","TRIBAL_CODE","FACILITY_ID","UNIT_ID","REL_POINT_ID","PROCESS_ID","AGY_FACILITY_ID","AGY_UNIT_ID",                
            "AGY_REL_POINT_ID","AGY_PROCESS_ID","SCC","POLID","ANN_VALUE","ANN_PCT_RED","FACILITY_NAME","ERPTYPE","STKHGT",
            "STKDIAM","STKTEMP","STKFLOW","STKVEL","NAICS","LONGITUDE","LATITUDE","LL_DATUM","HORIZ_COLL_MTHD","DESIGN_CAPACITY",
            "DESIGN_CAPACITY_UNITS","REG_CODES","FAC_SOURCE_TYPE","UNIT_TYPE_CODE","CONTROL_IDS","CONTROL_MEASURES",
            "CURRENT_COST","CUMULATIVE_COST","PROJECTION_FACTOR","SUBMITTER_FAC_ID","CALC_METHOD","DATA_SET_ID","FACIL_CATEGORY_CODE",
            "ORIS_FACILITY_CODE","ORIS_BOILER_ID","IPM_YN","CALC_YEAR","DATE_UPDATED","FUG_HEIGHT","FUG_WIDTH_XDIM","FUG_LENGTH_YDIM",
            "FUG_ANGLE","ZIPCODE","ANNUAL_AVG_HOURS_PER_YEAR","JAN_VALUE", "FEB_VALUE", "MAR_VALUE",
            "APR_VALUE", "MAY_VALUE", "JUN_VALUE", "JUL_VALUE", "AUG_VALUE", "SEP_VALUE", "OCT_VALUE", "NOV_VALUE", "DEC_VALUE",
            "JAN_PCTRED", "FEB_PCTRED", "MAR_PCTRED", "APR_PCTRED", "MAY_PCTRED", "JUN_PCTRED", "JUL_PCTRED", "AUG_PCTRED",
            "SEP_PCTRED", "OCT_PCTRED", "NOV_PCTRED", "DEC_PCTRED", "COMMENT"    
        ])
        
        df[!, :FIPS] = begin
            fips_transformed = String[]
            for fips in df[!, :FIPS]
                fips_str = string(fips)
                if length(fips_str) == 6
                    fips_str = fips_str[2:end]
                end
                push!(fips_transformed, lpad(fips_str, 5, '0'))
            end
            fips_transformed
        end

        
        df[!, :SCC] = [lpad(scc, 10, '0') for scc in string.(df[!, :SCC])]
        
        df.ANN_VALUE = df.ANN_VALUE * tonperyear
        
        df.JAN_VALUE = df.JAN_VALUE * tonpermonth
        df.FEB_VALUE = df.FEB_VALUE * tonpermonth
        df.MAR_VALUE = df.MAR_VALUE * tonpermonth
        df.APR_VALUE = df.APR_VALUE * tonpermonth
        df.MAY_VALUE = df.MAY_VALUE * tonpermonth
        df.JUN_VALUE = df.JUN_VALUE * tonpermonth
        df.JUL_VALUE = df.JUL_VALUE * tonpermonth
        df.AUG_VALUE = df.AUG_VALUE * tonpermonth
        df.SEP_VALUE = df.SEP_VALUE * tonpermonth
        df.OCT_VALUE = df.OCT_VALUE * tonpermonth
        df.NOV_VALUE = df.NOV_VALUE * tonpermonth
        df.DEC_VALUE = df.DEC_VALUE * tonpermonth
        
        df.STKHGT = df.STKHGT * foot
        df.STKDIAM = df.STKDIAM * foot
        df.STKTEMP = kelvin.(df.STKTEMP)
        df.STKFLOW = df.STKFLOW * foot * foot * foot / u"s"
        df.STKVEL = df.STKVEL * foot / u"s"
        
        # Fill in missing parameters.
        # Commented out for now.
        #flowmissing = ismissing.(df.STKFLOW)
        #df.STKFLOW[flowmissing] .= df.STKVEL[flowmissing] .* (df.STKDIAM[flowmissing] .* df.STKDIAM[flowmissing] / 4 * π)
        
        #velmissing = ismissing.(df.STKVEL)
        #df.STKVEL[velmissing] .= df.STKVEL[velmissing] ./ (df.STKDIAM[velmissing] .* df.STKDIAM[velmissing] / 4 * π)
        
        return new(df)
    end
end

strip_missing(x) = ismissing(x) ? "" : strip(x)

# getCountry returns the Country associated with this code.
function getCountry(code)
    countryMap = Dict(
        "0" => "USA",
        "1" => "Canada",
        "2" => "Mexico",
        "3" => "Cuba",
        "4" => "Bahamas",
        "5" => "Haiti",
        "6" => "Dominican Republic",
        "7" => "Global"
    )
    return get(countryMap, code, "Unknown")
end

# read_grid reads grid file data.
function read_grid(file_path)
    data = DataFrame(COUNTRY=String[], FIPS=String[], SCC=String[], Surrogate=String[])
    open(file_path) do file
        for line in eachline(file)
            if startswith(line, '#') || !contains(line, ';')
                continue
            end

            parts = split(line, ';', limit=3)

            if length(parts) >= 3
                surrogate_part = split(parts[3], '!', limit=2)[1]
                fips = parts[1]
                scc = parts[2]
                surrogate = strip(surrogate_part)

                country = "USA"
                if length(fips) == 6
                    country = getCountry(fips[1:1])
                    fips = fips[2:end]
                end

                if length(scc) == 8
                    scc = "00" * scc
                end

                push!(data, (COUNTRY=country, FIPS=fips, SCC=scc, Surrogate=surrogate))
            end
        end
    end
    return data
end

# getShapefilePath gets the shapefile path of a DataShapefile or WeightShapefile.
function getShapefilePath(baseDir::String, ShapefileName::String, CheckShapefiles::Bool)::String
    if CheckShapefiles
        for (root, dirs, files) in walkdir(baseDir)
            for file in files
                if file == ShapefileName * ".shp"
                    return joinpath(root, file)
                end
            end
        end
        @warn "Shapefile not found: $(joinpath(baseDir, ShapefileName * ".shp"))"
        return ""
    else
        return joinpath(baseDir, ShapefileName * ".shp")
    end
end

# validateShapefile validates a shapefile.
function validateShapefile(shapefilePath::String)
    if shapefilePath != ""
        try
            shp = open(shapefilePath)
            close(shp)
            return true
        catch e
            @warn "Failed to open shapefile: $shapefilePath; Error: $e"
            return false
        end
    end
    return false
end

# readSrgSpecSMOKE reads a SMOKE formatted spatial surrogate specification file.
# Results are returned as a map of surrogate specifications.
function readSrgSpecSMOKE(SrgSpec::String, SrgShapefileDirectory::String, CheckShapefiles::Bool=false)
    
    df = CSV.File(SrgSpec; comment="#", header=1, delim=',', quotechar='"', escapechar='"') |> DataFrame

    srgs = SurrogateSpec[]

    for row in eachrow(df)
        WeightColumns = String[]
        WeightFactors = Float64[]
        MergeNames = String[]
        MergeMultipliers = Float64[]
        Region = get(row, 1, "") |> strip_missing
        Name = get(row, 2, "") |> strip_missing
        Code = get(row, 3, 0)
        DataShapefile = get(row, 4, "") |> strip_missing
        DataAttribute = get(row, 5, "") |> strip_missing
        WeightShapefile = get(row, 6, "") |> strip_missing
        WeightAttribute = get(row, 7, "") |> strip_missing
        WeightFunction = get(row, 8, "") |> strip_missing
        FilterFunction = get(row, 9, "") |> strip_missing
        MergeFunction = get(row, 10, "") |> strip_missing
        BackupSurrogateNames = [
            strip(coalesce(get(row, 11, missing), "")), 
            strip(coalesce(get(row, 12, missing), "")), 
            strip(coalesce(get(row, 13, missing), ""))
        ]
        BackupSurrogateNames = filter(s -> !isempty(s), BackupSurrogateNames)
        Details = get(row, 14, "") |> strip_missing

        if WeightAttribute != "none" && WeightAttribute != ""
            push!(WeightColumns, WeightAttribute)
            push!(WeightFactors, 1.0)
        end

        if WeightFunction != ""
            WeightFunctionParts = split(WeightFunction, "+")
            for wf in WeightFunctionParts
                mulFunc = split(wf, "*")
                if length(mulFunc) == 1
                    push!(WeightColumns, strip(mulFunc[1]))
                    push!(WeightFactors, 1.0)
                elseif length(mulFunc) == 2
                    factor = tryparse(Float64, strip(mulFunc[1]))
                    if isnothing(factor)
                        error("Invalid weight factor in weight function: $wf")
                    end
                    push!(WeightColumns, strip(mulFunc[2]))
                    push!(WeightFactors, factor)
                else
                    error("Invalid weight function format: $wf")
                end
            end
        end

        if MergeFunction != "none" && MergeFunction != ""
            parts = split(MergeFunction, "+")
            for part in parts
                components = split(part, "*")
                if length(components) == 2
                    Name = strip(components[2])
                    val = tryparse(Float64, strip(components[1]))
                    if isnothing(val)
                        error("Invalid merge multiplier in merge function: $part")
                    end
                    push!(MergeNames, Name)
                    push!(MergeMultipliers, val)
                else
                    error("Invalid merge function format: $part")
                end
            end
        end

        if !isempty(DataShapefile)
            DataShapefile = getShapefilePath(SrgShapefileDirectory, String(DataShapefile), CheckShapefiles)
            isValidDataShapefile = validateShapefile(DataShapefile)
        end
            
        if !isempty(WeightShapefile)
            WeightShapefile = getShapefilePath(SrgShapefileDirectory, String(WeightShapefile), CheckShapefiles)
            isValidWeightShapefile = validateShapefile(WeightShapefile)
        end
        
        push!(srgs, SurrogateSpec(Region, Name, Code, DataShapefile, DataAttribute,
                                        WeightShapefile, Details, BackupSurrogateNames,
                                        WeightColumns, WeightFactors, FilterFunction,
                                        MergeNames, MergeMultipliers))
        
    end
    
    return srgs
end

# NewSpatialProcessor creates a new spatial processor.
function NewSpatialProcessor(srgSpecs::Vector{SurrogateSpec}, grids::GridDef, gridRef::DataFrame, inputSR::String, matchFullSCC::Bool)
    sp = SpatialProcessor(
        srgSpecs,
        grids,
        gridRef,
        inputSR,
        matchFullSCC,
        100,
        10
    )
    return sp
end

# NewPolygon creates a new polygon from a string of coordinates.
function NewPolygon(polygon_str::String, inputSR::String, outputSR::String)
    points = Point{2, Float64}[]
    regex = r"X:(-?\d+(\.\d+)?(e[+\-]?\d+)?) Y:(-?\d+(\.\d+)?(e[+\-]?\d+)?)"
    trans = Proj4.Transformation(inputSR, outputSR)
    matches = collect(eachmatch(regex, polygon_str))
    for match in matches
        x1 = parse(Float64, match.captures[1])
        y1 = parse(Float64, match.captures[4])
        x, y = trans([x1, y1])
        push!(points, Point(x, y))
    end
    return Polygon(points)
end


# NewGridIrregular creates a new irregular grid.
function NewGridIrregular(name::String, filepath::String, inputSR::String, outputSR::String)
    gridCells = Vector{LibGEOS.Polygon}()
    gridExtent = Vector{Vector{Tuple{Float64, Float64}}}()
    regex = r"X:(-?\d+(\.\d+)?(e[+\-]?\d+)?) Y:(-?\d+(\.\d+)?(e[+\-]?\d+)?)"
    trans = Proj4.Transformation(inputSR, outputSR)

    open(filepath, "r") do file
        content = read(file, String)
        polygon_strs = split(replace(content, '\n' => ""), "]] [[")

        for polygon_str in polygon_strs
            cleaned_str = replace(polygon_str, r"[\[\]]" => "")
            matches = collect(eachmatch(regex, cleaned_str))

            wkt_points_str = join([
                let (x1, y1) = (parse(Float64, match.captures[1]), parse(Float64, match.captures[4])),
                    (x, y) = trans([x1, y1])
                in
                    "$(x) $(y)"
                end for match in matches
            ], ", ")

            polygon_wkt = "POLYGON(($wkt_points_str))"
            polygon = LibGEOS.readgeom(polygon_wkt)
            push!(gridCells, polygon)

            polygonCoords = [Tuple(trans([parse(Float64, match.captures[1]), parse(Float64, match.captures[4])])) for match in matches]
            push!(gridExtent, polygonCoords)
        end
    end
    
    Nx = 1
    Ny = length(gridCells)
    
    return GridDef(name, Nx, Ny, outputSR, gridCells, gridExtent)
end


# setupSpatialProcessor reads in the necessary information to initialize
# a processor for spatializing emissions, and then does so.
function setupSpatialProcessor(c::Config)

    srgSpecs = readSrgSpecSMOKE(c.SrgSpec, c.SrgShapefileDirectory, true)

    gridRef = DataFrame(COUNTRY=String[], FIPS=String[], SCC=String[], Surrogate=String[])
    for f in c.f_gridRef
        gridRefTemp = read_grid(f)
        append!(gridRef, gridRefTemp)
    end

    grids = NewGridIrregular(c.GridName, c.GridFile, c.OutputSR, c.OutputSR)

    sp = NewSpatialProcessor(srgSpecs, grids, gridRef, c.InputSR, false)

    return sp, grids
end

# findCountyPolygon finds county polygon based on location and counties shapefile.
function findCountyPolygon(loc::String, countiesShapefile::String)
    fips_match = match(r"\d+", loc)
    fips_code = fips_match.match

    prj_path = replace(countiesShapefile, ".shp" => ".prj")
    tgt_crs = open(prj_path, "r") do file
        read(file, String)
    end
    dbf_path = replace(countiesShapefile, ".shp" => ".dbf")
    shapes = Shapefile.Table(countiesShapefile)
    dbf_data = DBFTables.Table(dbf_path)
    attributes = DataFrame(dbf_data)

    for (index, shape) in enumerate(shapes)
        geoid = string(attributes[index, :GEOID])

        if geoid == fips_code
            countyPolygon = GeoInterface.convert(LibGEOS, shape.geometry)
            return countyPolygon, tgt_crs
        end
    end

    return nothing, nothing
end


# GetIndex returns the row and column indices for a county or point in the grid.
function GetIndex(grid::GridDef, gridCRS::String, emissionType::String; loc::String = "", countiesShapefile::String = "", x::Float64 = NaN, y::Float64 = NaN)
    rows = Int[]
    cols = Int[]
    fracs = Float64[]
    inGrid = false
    coveredByGrid = false

    if emissionType == "point"
        if isnan(x) || isnan(y)
            println("For point emissions, x and y coordinates must be provided")
            return nothing
        end

        point_wkt = "POINT($x $y)"
        point = LibGEOS.readgeom(point_wkt)
        pointInCells = Dict()

        for (index, ext) in enumerate(grid.Extent)
            coords_str = join(["$(coord[1]) $(coord[2])" for coord in ext], ", ")
            cell_wkt = "POLYGON(($coords_str))"
            cellPolygon = LibGEOS.readgeom(cell_wkt)

            if LibGEOS.intersects(point, cellPolygon)
                push!(pointInCells, index => cellPolygon)
            end
        end

        numIntersectingCells = length(pointInCells)
        if numIntersectingCells > 0
            inGrid = true
            fraction = 1.0 / numIntersectingCells
            for (idx, _) in pointInCells
                push!(rows, idx)
                push!(cols, 1)
                push!(fracs, fraction)
            end
        end

        coveredByGrid = true
    
    elseif emissionType == "area"
        countyPolygon, tgtCRS = findCountyPolygon(loc, countiesShapefile)
        if countyPolygon === nothing || tgtCRS === nothing
            println("Could not find Polygon or target CRS for location: $loc")
            return nothing
        end
        transformer = Proj4.Transformation(gridCRS, tgtCRS)
        totalArea = LibGEOS.area(countyPolygon)

        for (index, ext) in enumerate(grid.Extent)
            transformed_coords = [transformer(coord) for coord in ext]
            transformed_coords_str = join(["$(x) $(y)" for (x, y) in transformed_coords], ", ")
            cell_wkt = "POLYGON(($transformed_coords_str))"
            cellPolygon = LibGEOS.readgeom(cell_wkt)

            if LibGEOS.intersects(countyPolygon, cellPolygon)
                intersection = LibGEOS.intersection(countyPolygon, cellPolygon)
                cellArea = LibGEOS.area(intersection)
                push!(fracs, cellArea / totalArea)
                push!(rows, index)
                push!(cols, 1)
            end
        end

        if length(rows) > 0
            inGrid = true
        end
        if sum(fracs) > 0.9999
            coveredByGrid = true
        end

    else
        println("Invalid emission type: $emissionType")
        return nothing
    end

    return IndexInfo(rows, cols, fracs, inGrid, coveredByGrid)
end


# recordToGrid allocates the reciever to the specifed grid without any spatial surrogate.
function recordToGrid(grid::GridDef, loc::String, countiesShapefile::String, srcCRS::String)
    rows, cols, fracs, inGrid, coveredByGrid = GetIndex(grid, loc, countiesShapefile, srcCRS)
    
    gridSrg = spzeros(Float64, grid.Ny, grid.Nx)
    
    if inGrid
        for i in eachindex(rows)
            gridSrg[rows[i], cols[i]] += fracs[i]
        end
    end
    
    return gridSrg, coveredByGrid, inGrid
end

# GridFactors returns the normalized fractions of emissions in each grid cell.
function GridFactors(record::DataFrameRow, sp::SpatialProcessor, countiesShapefile::String, srcCRS::String)
    loc = record.COUNTRY == "USA" ? "US" * record.FIPS : record.COUNTRY * record.FIPS
    
    return recordToGrid(sp.Grids, loc, countiesShapefile, srcCRS)
end

# uniqueCoordinates gets unique coordinates from records.
function uniqueCoordinates(recs::Dict{String, Vector{DataFrameRow}})
    uCoords = Set{Tuple{Float64, Float64}}()

    for (scc, rows) in recs
        for row in rows
            if haskey(row, :LONGITUDE) && haskey(row, :LATITUDE)
                lon_lat = (row.LONGITUDE, row.LATITUDE)
                push!(uCoords, lon_lat)
            end
        end
    end

    return uCoords
end

# uniqueLoc gets unique locations (Country+FIPS) from records.
function uniqueLoc(recs::Dict{String, Vector{DataFrameRow}})
    uLocs = Set{String}()

    for (scc, rows) in records
        for row in rows
            country_fips = row.COUNTRY * row.FIPS  
            push!(uLocs, country_fips)
        end
    end

    return uLocs
end

# writeEmis writes emissions data to a shapefile.
function writeEmis(filename::String, gd::GridDef, records::Dict{String, Vector{DataFrameRow}}, indexDict::Dict{Union{Tuple{Float64, Float64}, String}, Union{Nothing, IndexInfo}}, InMAP_crs::String, emissionType::String)    
    df = DataFrame(
        cellIndex = Int[],
        VOC = Union{Float64, Missing}[],
        NOX = Union{Float64, Missing}[],
        NH3 = Union{Float64, Missing}[],
        SO2 = Union{Float64, Missing}[],
        PM25 = Union{Float64, Missing}[],
        SCC = String[]
    )

    scc_index = Dict()

    for (scc, scc_records) in records
        for record in scc_records
            pol = Symbol(record.POLID)
            ann_value = record.ANN_VALUE
            locIdx = nothing
            
            if emissionType == "point"
                if haskey(record, :LONGITUDE) && haskey(record, :LATITUDE)
                    longitude, latitude = record.LONGITUDE, record.LATITUDE
                    locIdx = indexDict[(longitude, latitude)]
                end
            elseif emissionType == "area"
                country = record.COUNTRY == "USA" ? "US" : record.COUNTRY
                loc = country * record.FIPS
                locIdx = indexDict[loc]
            end

            if locIdx !== nothing
                for (idx, fraction) in zip(locIdx.rows, locIdx.fracs)
                    emission_share = ustrip(ann_value * fraction)
                    key = (idx, scc)

                    if haskey(scc_index, key)
                        row_index = scc_index[key]
                        current_value = ismissing(df[row_index, pol]) ? 0.0 : df[row_index, pol]
                        df[row_index, pol] = current_value + emission_share
                    else
                        new_row = DataFrame(
                            cellIndex = [idx],
                            VOC = missing, NOX = missing, NH3 = missing,
                            SO2 = missing, PM25 = missing,
                            SCC = [scc]
                        )
                        new_row[!, pol] = [emission_share]
                        append!(df, new_row)
                        scc_index[key] = size(df, 1)
                    end
                end
            end
        end
    end

    function format_float(value)
        if ismissing(value)
            return "0.0"
        else
            return @sprintf("%.10e", value)
        end
    end

    polygons = Vector{LibGEOS.Polygon}()
    attributes = DataFrame(
        cellIndex = Int[],
        VOC = String[], NOX = String[], NH3 = String[],
        SO2 = String[], PM25 = String[], SCC = String[]
    )

    for row in eachrow(df)
        if row.cellIndex in keys(gd.Cells)
            push!(polygons, gd.Cells[row.cellIndex])
            push!(attributes, (
                cellIndex = row.cellIndex,
                VOC = format_float(row.VOC),
                NOX = format_float(row.NOX),
                NH3 = format_float(row.NH3),
                SO2 = format_float(row.SO2),
                PM25 = format_float(row.PM25),
                SCC = row.SCC
            ))
        end
    end

    writer = Shapefile.Writer(polygons, attributes)
    Shapefile.write(filename, writer, force=true)

    prj_filename = replace(filename, r"\.shp$" => "") * ".prj"
    open(prj_filename, "w") do f
        write(f, InMAP_crs)
    end
end



writeEmis (generic function with 1 method)

### Configuration

Next, we need to specify which files we are going to load and how we are going to load them.
There are several different file formats, and we need to specify what format each of the files is in.
For now, we are focusing on the files that contain annual emissions (rather than daily or hourly emissions
or activity data).

In [48]:
readpoint(f) = FF10PointDataFrame(CSV.read(joinpath(f), DataFrame, comment="#"))
readnonpoint(f) = FF10NonPointDataFrame(CSV.read(joinpath(f), DataFrame, comment="#"))
readnonroad(f) = FF10NonRoadDataFrame(CSV.read(joinpath(f), DataFrame, comment="#"))
readonroad(f) = FF10OnRoadDataFrame(CSV.read(joinpath(f), DataFrame, comment="#"))

emisdirs = (
     "2019ge_cb6_19k/inputs/afdust" => (
        "2019ge_from_afdust_2017NEIpost_NONPOINT_20210129_04nov2021_v0.csv" => readnonpoint,
    ),
    "2019ge_cb6_19k/inputs/airports" => (
        "2019ge_from_airports_SmokeFlatFile_2019NEI_POINT_20210914_runway_apportion_25jan2022_nf_v1.csv" => readpoint,
    ),
    # Format is FF10_POINT although first comment is: #FORMAT=FF10_NONPOINT
    "2019ge_cb6_19k/inputs/canada_ag" => (
        "canada_MYR_2016_ag_animal_NH3_VOC_monthly_12km_aggregated_09jun2021_v0.csv" => readpoint,
        "canada_MYR_2016_ag_fertilizer_NH3_monthly_12km_16apr2021_v0.csv" => readpoint,
    ),
    "2019ge_cb6_19k/inputs/canada_og2D" => (
        "canada_MYR_2016_point_UOG_15jun2021_nf_v1.csv" => readpoint,
    ),
    #"2019ge_cb6_19k/inputs/cem" => (    
    #),
    "2019ge_cb6_19k/inputs/cmv_c1c2_12" => (
        "cmv_c1c2_2019_12US1_2019_CA_annual_11feb2022_v0.csv" => readpoint,
        "cmv_c1c2_2019_12US1_2019_MX_annual_11feb2022_v0.csv" => readpoint,
        "cmv_c1c2_2019_12US1_2019_US_annual_11feb2022_v0.csv" => readpoint,
    ),
    "2019ge_cb6_19k/inputs/cmv_c3_12" => (
        "cmv_c3_2019_masked_12US1_2019_CA_annual_10feb2022_v0.csv" => readpoint,
        "cmv_c3_2019_masked_12US1_2019_MX_annual_10feb2022_v0.csv" => readpoint,
        "cmv_c3_2019_masked_12US1_2019_US_annual_10feb2022_v0.csv" => readpoint,
    ),
    "2019ge_cb6_19k/inputs/fertilizer" => (
        "2019ge_fertilizer_NH3_monthly_23jun2022_v0.csv" => readnonpoint,
    ),
    "2019ge_cb6_19k/inputs/livestock" => (
        "2019ge_from_livestock_2017NEIpost_NONPOINT_20210129_29nov2021_nf_v1.csv" => readnonpoint,
    ),
    "2019ge_cb6_19k/inputs/nonpt" => (
        "2017NEIpost_NONPOINT_20210129_08oct2021_v2.csv" => readnonpoint,
    ),
    "2019ge_cb6_19k/inputs/nonroad" => (
        "2019_interpolation_texas_nonroad_29oct2021_v1.csv" => readnonroad,
        "2019ge_from_2017nei_nonroad_california_29oct2021_v0.csv" => readnonroad,
        "2019ge_nonroad_from_MOVES_aggSCC_29oct2021_v1.csv" => readnonroad, 
    ),
    "2019ge_cb6_19k/inputs/np_oilgas" => (
        "np_oilgas_2019ge_02mar2022_nf_v3.csv" => readnonpoint,
    ),
    "2019ge_cb6_19k/inputs/np_solvents" => (
        "nonVCPy_solvents_2017NEIpost_NONPOINT_20210129_15feb2022_nf_v1.csv" => readnonpoint,
        "solvents_2017NEIpost_NONPOINT_20210129_nonVCPy_polls_for2016v3_2019ge_15feb2022_v0.csv" => readnonpoint,
        "VCPy_solvents_2019ge_minuspoint_21feb2022_nf_v2.csv" => readnonpoint,
    ),
    "2019ge_cb6_19k/inputs/onroad" => (
        "2019ge_SMOKE_MOVES_MOVES3_AQ_fullHAP_30nov2021_v0.csv" => readonroad,
    ),
    "2019ge_cb6_19k/inputs/onroad_can" => (
        "canada_2019ge_projection_T3_onroad_monthly_14mar2022_v0.csv" => readonroad,
        "canada_2019ge_projection_T3_onroad_refueling_14mar2022_v0.csv" => readonroad,
    ),
    "2019ge_cb6_19k/inputs/onroad_mex" => (
        "Mexico_2019_onroad_MOVES_interpolated_22oct2021_v0.csv" => readonroad,
    ),
    "2019ge_cb6_19k/inputs/othafdust" => (
        "canada_MYR_2016_area_dust_27dec2020_v0.csv" => readnonpoint,
    ),
    "2019ge_cb6_19k/inputs/othar" => (
        "2019ge_proj_CEDS_from_Mexico_2016INEM_nonpoint_09nov2021_v0.csv" => readnonpoint,
         "2019ge_proj_CEDS_from_Mexico_2016INEM_nonroad_09nov2021_v0.csv" => readnonroad,
        "canada_2019ge_proj_from2016_T4_nonroad_monthly_15mar2022_v0.csv" => readnonroad,
        "canada_MYR_2016_area_other_27dec2020_v0.csv" => readnonpoint,
        "canada_MYR_2016_T5_rail_27dec2020_v0.csv" => readnonpoint,
    ),
    "2019ge_cb6_19k/inputs/othpt" => (
        "2019ge_proj_CEDS_from_Mexico_2016_point_20191209_06mar2023_nf_v3.csv" => readpoint,
        "canada_MYR_2016_point_CB6VOC_monthly_27dec2020_v0.csv" => readpoint,
        "canada_MYR_2016_point_nodust_noVOC_monthly_27dec2020_v0.csv" => readpoint,
        "canada_MYR_2016_point_UOG_15jun2021_nf_v2.csv" => readpoint,
        "canada_MYR_2016_point_VOC_INV_monthly_27dec2020_v0.csv" => readpoint,
        "canada_MYR_2016_T1_airports_monthly_27dec2020_v0.csv" => readpoint,
    ),
    "2019ge_cb6_19k/inputs/othptdust" => (
        "canada_MYR_2016_ag_animal_PM10_monthly_12km_27dec2020_v0.csv" => readpoint,
         "canada_MYR_2016_ag_animal_PM25_monthly_12km_27dec2020_v0.csv" => readpoint,
        "canada_MYR_2016_ag_harvest_monthly_12km_27dec2020_v0.csv" => readpoint,
        "canada_MYR_2016_ag_tillage_monthly_12km_27dec2020_v0.csv" => readpoint,
        "canada_MYR_2016_point_source_dust_monthly_27dec2020_v0.csv" => readpoint,
    ),
    "2019ge_cb6_19k/inputs/pt_oilgas" => (
        "2019ge_from_oilgas_2019NEI_SmokeFlatFile_POINT_20220325_calcyear2017_07apr2022_v0.csv" => readpoint,
        "oilgas_2019NEI_SmokeFlatFile_POINT_20220325_calcyear2019_06apr2022_v0.csv" => readpoint,
    ),
    "2019ge_cb6_19k/inputs/ptagfire" => (
        "ptinv_agfire_CONUS_2019ge_23dec2021_v0.csv" => readpoint,
        "ptinv_agfire_FL_2019ge_23dec2021_v0.csv" => readpoint,
    ),
    "2019ge_cb6_19k/inputs/ptegu" => (
        "egucems_2019NEI_SmokeFlatFile_POINT_20220325_07apr2022_v0.csv" => readpoint,
        "egunoncems_2019NEI_SmokeFlatFile_POINT_20220325_08apr2022_nf_v1.csv" => readpoint,
    ),
    "2019ge_cb6_19k/inputs/ptfire_othna" => (
        "ptinv_finn_CA_finn_2019ge_ff10_26oct2021_v0.csv" => readpoint,
        "ptinv_finn_MX_finn_2019ge_ff10_26oct2021_v0.csv" => readpoint,
        "ptinv_finn_ONA_finn_2019ge_ff10_26oct2021_v0.csv" => readpoint,
    ),
    "2019ge_cb6_19k/inputs/ptfire-rx" => (
        "ptinv_ptfire_FH_grassland_2019ge_23dec2021_v0.csv" => readpoint,
        "ptinv_ptfire_sf2_2019ge_bsp_16dec2021_nf_v1.csv" => readpoint,
        "ptinv_ptfire_sf2_2019ge_bsp_haps_16dec2021_nf_v1.csv" => readpoint,
    ),
    "2019ge_cb6_19k/inputs/ptfire-wild" => (
        "ptinv_ptfire_sf2_2019ge_bsp_16dec2021_nf_v2.csv" => readpoint,
        "ptinv_ptfire_sf2_2019ge_bsp_haps_16dec2021_nf_v2.csv" => readpoint,
    ),
    "2019ge_cb6_19k/inputs/ptnonipm" => (
        "2019_eto_ptnonipm_supplemental_07apr2022_nf_v1.csv" => readpoint,
        "nonegu_2019NEI_SmokeFlatFile_POINT_20220325_07apr2022_nf_v1.csv" => readpoint,
        "railyard_postprojection_2019NEI_SmokeFlatFile_POINT_20211221_05apr2022_nf_v1.csv" => readpoint,
    ),
    "2019ge_cb6_19k/inputs/rail" => (
        "2019ge_from_rail_2017NEIpost_NONPOINT_20210129_04jan2022_v0.csv" => readnonpoint,
    ),
    # CONUS Only
    "2019ge_cb6_19k/inputs/rwc" => (
        "rwc_2017NEIpost_NONPOINT_20210129_06apr2022_nf_v6.csv" => readnonpoint,
    ),    
);

In [5]:
# ptegu - for testing
readpoint(f) = FF10PointDataFrame(CSV.read(joinpath(f), DataFrame, comment="#"))
readnonpoint(f) = FF10NonPointDataFrame(CSV.read(joinpath(f), DataFrame, comment="#"))
readnonroad(f) = FF10NonRoadDataFrame(CSV.read(joinpath(f), DataFrame, comment="#"))
readonroad(f) = FF10OnRoadDataFrame(CSV.read(joinpath(f), DataFrame, comment="#"))

emisdirs = (
     
    "2019ge_cb6_19k/inputs/ptegu" => (
        "egucems_2019NEI_SmokeFlatFile_POINT_20220325_07apr2022_v0.csv" => readpoint,
        "egunoncems_2019NEI_SmokeFlatFile_POINT_20220325_08apr2022_nf_v1.csv" => readpoint,
    ),    
);

### Load emissions

Now we iterate through each of the files listed above and read them into dataframes. Each file contains a lot of data; however, we are only interested in the pollutant name, country name, FIPS, and the Source Classification Code (SCC) that describes the type of emitter, as well as the annual emissions rate, longitude (for point sources), and latitude (for point sources). Therefore, for now, we will only save these variables.

In [6]:
dfs = []

for (emisdir, data) ∈ emisdirs
    for (file, readfunc) ∈ data
        path = joinpath(dir, emisdir, file)
        df = readfunc(path)
        
        if readfunc === readpoint
            push!(dfs, df.df[!, [:POLID, :COUNTRY, :FIPS, :SCC, :ANN_VALUE, :LONGITUDE, :LATITUDE]])
        elseif readfunc === readnonpoint || readfunc === readnonroad || readfunc === readonroad
            area_df = df.df[!, [:POLID, :COUNTRY, :FIPS, :SCC, :ANN_VALUE]]
            area_df[!, :LONGITUDE] = missing
            area_df[!, :LATITUDE] = missing
            push!(dfs, area_df)
        else
            println("Unknown emission type")
        end
    end
end


### Results

Next, let's combine all the files together into one big dataframe.

In [7]:
emis = vcat(dfs...)

Row,POLID,COUNTRY,FIPS,SCC,ANN_VALUE,LONGITUDE,LATITUDE
,String15,String3,String,String,Quantity…,Float64,Float64
1,PM25-PRI,US,25027,0020100201,0.000260206 kg s^-1,-72.0151,42.1124
2,PM10-PRI,US,25027,0020100201,0.000260206 kg s^-1,-72.0151,42.1124
3,PM10-PRI,US,21049,0020100101,3.12612e-7 kg s^-1,-84.1025,37.8824
4,PM25-PRI,US,21049,0020100101,2.84943e-7 kg s^-1,-84.1025,37.8824
5,PM10-PRI,US,21049,0020100101,4.39207e-7 kg s^-1,-84.1025,37.8824
6,PM25-PRI,US,21049,0020100101,4.00333e-7 kg s^-1,-84.1025,37.8824
7,PM25-PRI,US,18097,0010200601,7.76502e-5 kg s^-1,-86.1665,39.7629
8,PM10-PRI,US,18097,0010200601,7.76502e-5 kg s^-1,-86.1665,39.7629
9,PM25-PRI,US,27123,0020100201,5.92593e-5 kg s^-1,-93.1116,44.9315


A lot of the records above are duplicates; let's combine them all. 

In [8]:
emis_grouped = sort(combine(groupby(emis, [:POLID, :COUNTRY, :FIPS, :SCC, :LONGITUDE, :LATITUDE]), :ANN_VALUE=>sum=>:ANN_VALUE), :ANN_VALUE, rev=true)

Row,POLID,COUNTRY,FIPS,SCC,LONGITUDE,LATITUDE,ANN_VALUE
,String15,String3,String,String,Float64,Float64,Quantity…
1,CO2,US,18173,0010100202,-87.3329,37.9147,95.6968 kg s^-1
2,CO2,US,18173,0010100203,-87.3333,37.9145,51.9336 kg s^-1
3,CO2,US,06029,0020100201,-119.47,35.2802,43.1176 kg s^-1
4,CO2,US,18089,0020100201,-87.4775,41.6739,41.6498 kg s^-1
5,CO2,US,48167,0020200203,-94.9332,29.3777,36.5314 kg s^-1
6,CO2,US,47075,0020100201,-89.3925,35.6499,35.107 kg s^-1
7,CO2,US,48167,0020200203,-94.9322,29.3777,34.9882 kg s^-1
8,CO2,US,48167,0020200203,-94.9327,29.3777,29.4392 kg s^-1
9,CO2,US,33007,0010100903,-71.1752,44.4712,25.6217 kg s^-1


Next, we filter the grouped emissions records to include only the rows where POLID is a key in the Pollutants dictionary, and we group the emissions records by their SCC.

In [9]:
filtered_emis_grouped = filter(row -> row.POLID in keys(Pollutants), emis_grouped)
for row in eachrow(filtered_emis_grouped)
    row.POLID = Pollutants[row.POLID]
end

records = Dict{String, Vector{DataFrameRow}}()

for row in eachrow(filtered_emis_grouped)
    scc = row.SCC
    if haskey(records, scc)
        push!(records[scc], row)
    else
        records[scc] = [row]
    end
end


Now, we will perform spatial and temporal processing on these emissions and save the results to a shapefile.

In [10]:
# This is an example for ptegu. We will update it to loop over all sectors.

c = Config([
"C:/Users/hemamipo/OneDrive - University of Illinois - Urbana/Research-UIUC/Tessum Group/Tasks/NEI_Processor/2019nei/ge_dat/gridding/agref_us_2017platform_13jan2022_nf_v12.txt",
"C:/Users/hemamipo/OneDrive - University of Illinois - Urbana/Research-UIUC/Tessum Group/Tasks/NEI_Processor/2019nei/ge_dat/gridding/amgref_can2015_mex2010_for2016beta_09mar2021_nf_v5.txt",
"C:/Users/hemamipo/OneDrive - University of Illinois - Urbana/Research-UIUC/Tessum Group/Tasks/NEI_Processor/2019nei/ge_dat/gridding/mgref_onroad_MOVES3_23sep2021_v1.txt"
],
"C:\\Users\\hemamipo\\OneDrive - University of Illinois - Urbana\\Research-UIUC\\Tessum Group\\Tasks\\NEI_Processor\\Scripts\\surrogate_specification_2019.csv",
"C:\\Users\\hemamipo\\Downloads\\tmp2\\NEI_Processor\\nei2019\\Shapefiles",
"+proj=longlat",
"+proj=lcc +lat_1=33 +lat_2=45 +lat_0=40 +lon_0=-97 +x_0=0 +y_0=0 +a=6370997 +b=6370997 +to_meter=1",
"C:\\Users\\hemamipo\\OneDrive - University of Illinois - Urbana\\Research-UIUC\\Tessum Group\\Tasks\\NEI_Processor\\gridCells.txt",
"InMAP",
"C:/Users/hemamipo/OneDrive - University of Illinois - Urbana/Research-UIUC/Tessum Group/Tasks/NEI_Processor/2019nei/cb_2019_us_county_500k/cb_2019_us_county_500k.shp",
"C:/Users/hemamipo/OneDrive - University of Illinois - Urbana/Research-UIUC/Tessum Group/Tasks/NEI_Processor/Emission Shapefiles/2019"
)

InMAP_crs = """
PROJCS[\"Lambert_Conformal_Conic\",GEOGCS[\"GCS_unnamed ellipse\",DATUM[\"D_unknown\",SPHEROID[\"locIndexUnknown\",6370997.000000,0]],PRIMEM[\"Greenwich\",0],UNIT[\"Degree\",0.017453292519943295]],PROJECTION[\"Lambert_Conformal_Conic\"],PARAMETER[\"standard_parallel_1\",33],PARAMETER[\"standard_parallel_2\",45],PARAMETER[\"latitude_of_origin\",40],PARAMETER[\"central_meridian\",-97],PARAMETER[\"false_easting\",0],PARAMETER[\"false_northing\",0],UNIT[\"Meter\",1]]
"""

sp, gd = setupSpatialProcessor(c)

uCoordinates = uniqueCoordinates(records)
gdCRS = "+proj=lcc +lat_1=33 +lat_2=45 +lat_0=40 +lon_0=-97 +x_0=0 +y_0=0 +a=6370997 +b=6370997 +to_meter=1"
emisCRS = "EPSG:4326" 

transformer = Proj4.Transformation(emisCRS, gdCRS)

uCoordinates = uniqueCoordinates(records)

locIndex = Dict{Union{Tuple{Float64, Float64}, String}, Union{IndexInfo, Nothing}}()

for (longitude, latitude) in uCoordinates
    x, y = transformer([latitude, longitude])
    idx = GetIndex(gd, gdCRS, "point",x=x, y=y)
    locIndex[(longitude, latitude)] = idx
end

writeEmis(c.EmisShp * "/ptegu.shp", gd, records, locIndex, InMAP_crs, "point")


440

Our next steps will be to perform more spatial and temporal processing on these emissions. To do this, we will pull in the EPA's spatial surrogate shapefiles to allocate county-level emissions to locations within each county, and we will leverage the daily and hourly emission files above plus temporal surrogate information to allocate the emissions to times during the year. Finally, we will leverage emissions factor and meteorology information to convert the mobile and fire activity data above into emissions.

Stay tuned!